In [ ]:
import os
import json
import pandas as pd

In [ ]:
BASE_DIR = './run_time_record'
EXPECTED_FILES = 26

# 1. Collect JSON files and assert count
json_files = sorted(f for f in os.listdir(BASE_DIR) if f.endswith('.json'))
assert len(json_files) == EXPECTED_FILES, f"Expected {EXPECTED_FILES} JSON files, found {len(json_files)}"

In [ ]:
def parse_times(filepath):
    data = json.load(open(filepath, 'r', encoding='utf-8'))
    # Single-entry times
    const_t = data['const-var-time']['runTime']
    action_t = data['action-time']['runTime']
    nl_t = data['nl-annotation-time']['runTime']
    # Multi-entry
    codegen = [v['runTime'] for k,v in data.items() if k.startswith('codegen-time_')]
    verify  = [v['runTime'] for k,v in data.items() if k.startswith('verification-time_')]
    refine  = [(int(k.split('_')[1]), v['runTime'])
               for k,v in data.items() if k.startswith('refine-time_')]
    # Stats
    n_cg = len(codegen)
    tot_cg = sum(codegen)
    avg_cg = tot_cg / n_cg if n_cg else 0.0
    rounds = [i for i,_ in refine]
    tot_rounds = max(rounds) if rounds else 0
    attempts = len(refine)
    tot_ref_t = sum(t for _,t in refine)
    avg_ref_t = tot_ref_t / attempts if attempts else 0.0
    n_v = len(verify)
    tot_v = sum(verify)
    avg_v = tot_v / n_v if n_v else 0.0
    return {
        'const_var_time': const_t,
        'action_time': action_t,
        'nl_annotation_time': nl_t,
        'num_codegen': n_cg,
        'total_codegen_time': tot_cg,
        'avg_codegen_time': avg_cg,
        'total_refine_rounds': tot_rounds,
        'total_refine_attempts': attempts,
        'total_refine_time': tot_ref_t,
        'avg_refine_time': avg_ref_t,
        'num_verifications': n_v,
        'total_verif_time': tot_v,
        'avg_verif_time': avg_v
    }

In [ ]:
# 2. Build per-system DataFrame
records = []
for fname in json_files:
    sys = os.path.splitext(fname)[0]
    stats = parse_times(os.path.join(BASE_DIR, fname))
    stats['system'] = sys
    records.append(stats)
df = pd.DataFrame(records)

In [ ]:
# 3. Compute overall averages
overall = df.mean(numeric_only=True).to_frame().T

In [ ]:
# 4a) Compute detailed summary stats for all time columns
time_cols = [
    'const_var_time',
    'action_time',
    'nl_annotation_time',
    'total_codegen_time',
    'avg_codegen_time',
    'total_refine_time',
    'avg_refine_time',
    'total_verif_time',
    'avg_verif_time'
]

# describe gives min, 25%, 50%, 75%, max, mean, std
detailed_stats = (
    df[time_cols]
      .describe(percentiles=[0.25, 0.5, 0.75])
      .T
      .loc[:, ['min','25%','50%','75%','max','mean','std']]
)

In [ ]:
# 4b) Then in your ExcelWriter block, dump it to its own sheet
with pd.ExcelWriter('time_summary.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='PerSystem', index=False)
    overall.to_excel(writer, sheet_name='Averages', index=False)
    detailed_stats.to_excel(writer, sheet_name='TimeDistributions')

print("✅  Saved time_summary.xlsx with sheets 'PerSystem', 'Averages', and 'TimeDistributions'")